# Erdos-Szekeres Happy Ending problem

In [ ]:
#@title Data and verification


import numpy as np
import random
import math
import numba

njit = numba.njit


# --- Numba Version (from above) ---
@numba.jit(nopython=True, fastmath=True)
def orientation_numba(p, q, r):
  val = (q[0] - p[0]) * (r[1] - q[1]) - (q[1] - p[1]) * (r[0] - q[0])
  if val > 1e-9:
    return 1
  if val < -1e-9:
    return -1
  return 0


@numba.jit(nopython=True, fastmath=True)
def calculate_score_numba(points_np: np.ndarray, n: int) -> int:
  num_points = len(points_np)
  if num_points < n or n < 3:
    return 0
  if num_points != 2 ** (n - 2) + 1:
    return -np.inf

  # Proximity check: Add a large penalty if any two points are too close.
  # This prevents floating-point issues in orientation calculations.
  # We use squared distance to avoid the expensive sqrt operation.
  min_dist_sq = 1e-12  # Corresponds to a minimum distance of 1e-6
  for i in range(num_points):
    for j in range(i + 1, num_points):
      dist_sq = (points_np[i, 0] - points_np[j, 0])**2 + \
                (points_np[i, 1] - points_np[j, 1])**2
      if dist_sq < min_dist_sq:
        return -1e15  # Return a large penalty.

  collinearity_threshold = 1e-9
  for i in range(num_points):
    for j in range(i + 1, num_points):
      for k in range(j + 1, num_points):
        p1 = points_np[i]
        p2 = points_np[j]
        p3 = points_np[k]
        # This formula calculates a value proportional to the signed area of the triangle.
        area = p1[0] * (p2[1] - p3[1]) + p2[0] * (p3[1] - p1[1]) + p3[0] * (p1[1] - p2[1])
        if abs(area) < collinearity_threshold:
          print(area, p1, p2, p3)
          return -1e15+1000 # Return a large penalty

  P = points_np
  total_ngons = 0
  for i in range(num_points):
    pivot = P[i]
    Q_buffer = np.zeros((num_points - 1 - i, 2))
    count = 0
    for j in range(i + 1, num_points):
      Q_buffer[count] = P[j]
      count += 1
    if count == 0:
      continue
    Q_unsorted = Q_buffer[:count]
    angles = np.zeros(len(Q_unsorted))
    for j in range(len(Q_unsorted)):
      angles[j] = math.atan2(
          Q_unsorted[j, 1] - pivot[1], Q_unsorted[j, 0] - pivot[0]
      )
    sort_indices = np.argsort(angles)
    Q = Q_unsorted[sort_indices]
    M = len(Q)
    if M < n - 1:
      continue
    dp = np.zeros((n + 1, M, M), dtype=np.int64)
    for j in range(M):
      for k in range(j):
        if orientation_numba(pivot, Q[k], Q[j]) > 0:
          dp[3, k, j] = 1
    for length in range(4, n + 1):
      for j in range(M):
        for k in range(j):
          for prev_k in range(k):
            if (
                dp[length - 1, prev_k, k] > 0
                and orientation_numba(Q[prev_k], Q[k], Q[j]) > 0
            ):
              dp[length, k, j] += dp[length - 1, prev_k, k]
    for j in range(M):
      for k in range(j):
        if dp[n, k, j] > 0 and orientation_numba(Q[k], Q[j], pivot) > 0:
          total_ngons += dp[n, k, j]
  return -total_ngons


def calculate_score(points: list[list[float]], n: int) -> int:
  sorted_points = sorted([tuple(p) for p in points])
  points_np = np.array(sorted_points, dtype=np.float64)
  return calculate_score_numba(points_np, n)


# dummy calculate_score call
calculate_score(np.array([[0,0]]), 2)

points_7 = placed_points_7 = np.array([[0.4468800783395808 , 0.3821544338354585 ], [0.44690869175949194, 0.3822238345412346 ], [0.377886716576163 , 0.3822765194693299 ], [0.44828927694234205, 0.3831044215220069 ], [0.44827090457772534, 0.38217629987035145], [0.3758225259601966 , 0.38121024439494877], [0.42806239823065817, 0.3858085643025208 ], [0.4256139368723774 , 0.3842034795399387 ], [0.4482549869365213 , 0.3822350104577767 ], [0.45456668058968147, 0.3837467196340445 ], [0.41858324923435963, 0.39390398937305976], [0.4276354975361686 , 0.38641468988047584], [0.42874345967357824, 0.38520339290559313], [0.44557294794557906, 0.3819333733279178 ], [0.45139322269098064, 0.37772945634745764], [0.4506126565311825 , 0.37768244525380973], [0.41733763122941786, 0.39293428649245016], [0.4500985884329095 , 0.3776151034874538 ], [0.44555158528492467, 0.3819657030872966 ], [0.4171845762459504 , 0.40039487200942797], [0.4500836805466427 , 0.37672396158287597], [0.3445722268574353 , 0.38873881816729156], [0.4183740323718786 , 0.3936424488835749 ], [0.45030111731132083, 0.3692195961795395 ], [0.4459843568155286 , 0.3805997680302024 ], [0.45733895861362456, 0.3766544440896039 ], [0.42598429709461566, 0.38473714444220625], [0.42604409003138466, 0.38499041071886575], [0.4495810136473284 , 0.38312013768379904], [0.37501638476075205, 0.3849939120047631 ], [0.37880688011251024, 0.3842880798629318 ], [0.3791005905975771 , 0.3839260411363063 ], [0.48558346208312336, 0.3700415511075106 ]])
points_8 = placed_points_8 = np.array([[0.7323571334193899 , 0.7684520225747564 ], [0.7331730698843125 , 0.767962770605042 ], [0.4223833145875395 , 0.5599197551783185 ], [0.4268014585993596 , 0.5492343659470436 ], [0.33430138846644736, 0.4757311138512364 ], [0.3383040668782688 , 0.46736545747336333], [0.6336804783815286 , 0.3222519878146729 ], [0.2903851486051504 , 0.4744784215838625 ], [0.3306378555898603 , 0.47995747542844874], [0.33868122911495496, 0.4673002520772869 ], [0.3352761213940171 , 0.4753478716320103 ], [0.1533308458028678 , 0.9407330005638428 ], [0.21138213186946697, 0.8983836152153677 ], [0.2600982482761151 , 0.45212760433437976], [0.3929264361823265 , 0.5765624764487487 ], [0.7330644952004246 , 0.7748412518022245 ], [0.633689565048656 , 0.3222950819601261 ], [0.4223193692608903 , 0.5598776477160156 ], [0.37736627941654227, 0.5950806119354964 ], [0.35311864206142296, 0.6059401685281832 ], [0.28810886754354637, 0.4636376476314433 ], [0.3783390061545638 , 0.5948400941374824 ], [0.6499532465313242 , 0.3322502884437739 ], [0.7345191227021849 , 0.7673109424121698 ], [0.3872258457866015 , 0.585557694270539 ], [0.6463350468692588 , 0.3318739654419432 ], [0.29007319031464796, 0.47476708501715026], [0.7323958333575616 , 0.768863025584097 ], [0.37978077859662684, 0.5955163404760805 ], [0.7345619741974515 , 0.7672970450889767 ], [0.7323690997974068 , 0.7683769146796294 ], [0.2609419998932585 , 0.45212017385488934], [0.25552960099351696, 0.4514592081131921 ], [0.426866204338735 , 0.5492718970833148 ], [0.2619375370703345 , 0.45167607042371666], [0.15331105316285845, 0.9407541576842592 ], [0.3783385015115275 , 0.5949505612575334 ], [0.6510753392609311 , 0.3323160295135607 ], [0.6359282063851999 , 0.32839570036531046], [0.6531537028467094 , 0.3319718883602552 ], [0.33130217123463684, 0.47947003848905784], [0.3798638956486957 , 0.5953468543396011 ], [0.3873116009902781 , 0.5849328788987167 ], [0.28974781346388595, 0.474843509813933 ], [0.42352312528007224, 0.5554113130368437 ], [0.39285689376123867, 0.5765658151644145 ], [0.3773768056720833 , 0.5950514229591739 ], [0.3872845939741692 , 0.5855423917816529 ], [0.387415680928708 , 0.5855892132006659 ], [0.26104521923700574, 0.45211803750389795], [0.3893855301750177 , 0.577607811140417 ], [0.17739215579737264, 0.9205077681053561 ], [0.3532860375114724 , 0.6044928854083396 ], [0.48444176388389293, 0.5323001719139354 ], [0.3321182171826289 , 0.479381208427889 ], [0.38725930097479583, 0.5849212636780967 ], [0.4234667627701185 , 0.5553758382150744 ], [0.7329870229233744 , 0.7747423379748621 ], [0.2903975478711358 , 0.4736900856768733 ], [0.21898981972360063, 0.8944085744083489 ], [0.26253142438584626, 0.45128481320840624], [0.4844248922126498 , 0.5322404955457067 ], [0.38714862645362563, 0.5849221226236265 ], [0.48716741782595757, 0.5403111733825767 ], [0.2626564754103592 , 0.4511074646705314 ]])

print(f"The number of convex 7-gons in the n=7 construction: {-calculate_score(points_7, 7)}")
print(f"The number of convex 8-gons in the n=8 construction: {-calculate_score(points_8, 8)}")


In [ ]:
#@title Plotting

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import itertools

def orientation_py(p, q, r):
    """Pure Python orientation test for use in non-Numba functions."""
    p_np, q_np, r_np = np.array(p), np.array(q), np.array(r)
    return orientation_numba(p_np, q_np, r_np)


def find_convex_ngons(points: np.ndarray, n: int) -> list[list[tuple[float, float]]]:
    """
    Finds all unique convex n-gons in a set of points.

    This function adapts the dynamic programming approach from the original
    problem description. Instead of merely counting polygons, it reconstructs
    the vertex paths for each valid n-gon found.

    Args:
        points: A NumPy array of shape (num_points, 2).
        n: The number of vertices in the target polygon (e.g., 7 for a heptagon).

    Returns:
        A list of polygons. Each polygon is a list of its vertex coordinates.
    """
    # Sort points lexicographically to ensure a canonical starting order.
    P = sorted([tuple(p) for p in points])
    num_points = len(P)
    if num_points < n:
        return []

    all_polygons = []

    # Iterate through each point, treating it as a potential 'pivot'.
    for i in range(num_points):
        pivot = P[i]

        # Create a list 'Q' of all points that come after the pivot in the
        # sorted list, then sort them by angle around the pivot.
        Q = sorted([p for j, p in enumerate(P) if j > i],
                   key=lambda p: math.atan2(p[1] - pivot[1], p[0] - pivot[0]))

        M = len(Q)
        if M < n - 1:
            continue

        # dp[length][j] will store lists of convex chains of (length-1) vertices
        # from Q that end at point Q[j].
        dp = {}

        # Initialize for length 3 (triangles).
        # A chain is a tuple of indices into Q.
        dp[3] = [[] for _ in range(M)]
        for j in range(M):
            for k in range(j):
                # A convex 3-gon is pivot -> Q[k] -> Q[j] with a left turn at Q[k].
                if orientation_py(pivot, Q[k], Q[j]) > 0:
                    dp[3][j].append((k, j))

        # Build up polygons of length 4 to n.
        for length in range(4, n + 1):
            dp[length] = [[] for _ in range(M)]
            for j in range(M):
                for k in range(j):
                    # For each chain of length-1 ending at Q[k]...
                    for chain in dp[length - 1][k]:
                        # ...check if adding Q[j] preserves convexity.
                        prev_k = chain[-2] # The point before Q[k] in the chain.
                        if orientation_py(Q[prev_k], Q[k], Q[j]) > 0:
                            dp[length][j].append(chain + (j,))

        # Check the fully formed n-gons for the final closing turn.
        if n in dp:
            for chain in itertools.chain.from_iterable(dp[n]):
                k = chain[-2]
                j = chain[-1]
                # Check the turn from the last edge back to the pivot.
                if orientation_py(Q[k], Q[j], pivot) > 0:
                    # If valid, construct the polygon from the pivot and the chain indices.
                    polygon = [pivot] + [Q[idx] for idx in chain]
                    all_polygons.append(polygon)

    # Remove duplicate polygons found from different pivots.
    unique_polygons = []
    seen_polygons = set()
    for poly in all_polygons:
        canonical_poly = tuple(sorted(poly))
        if canonical_poly not in seen_polygons:
            seen_polygons.add(canonical_poly)
            unique_polygons.append(poly)
    print(f"Found {len(unique_polygons)} unique convex {n}-gon(s).")
    print(f"Convex polygons: {unique_polygons}")
    return unique_polygons


def plot_with_ngons(points: np.ndarray, n: int):
    """
    Creates a scatter plot of points and highlights all found convex n-gons.
    """
    fig, ax = plt.subplots(figsize=(10, 10))

    print(f"Searching for convex {n}-gons...")
    ngons = find_convex_ngons(points, n)
    print(f"Found {len(ngons)} unique convex {n}-gon(s).")

    for ngon in ngons:
        poly_patch = patches.Polygon(np.array(ngon), closed=True,
                                     edgecolor='red', facecolor='#FF6347',
                                     alpha=0.4, zorder=1, linewidth=1.5)
        ax.add_patch(poly_patch)

    ax.scatter(points[:, 0], points[:, 1], c='#0033A0', s=50, zorder=2,
               alpha=0.9, edgecolors='k', linewidth=0.5)

    ax.set_title(f'Point Configuration Highlighting Convex {n}-gons ({len(points)} points)')
    ax.set_xlabel('X-coordinate')
    ax.set_ylabel('Y-coordinate')
    ax.grid(True, linestyle='--', alpha=0.6)

    x_min, x_max = points[:, 0].min(), points[:, 0].max()
    y_min, y_max = points[:, 1].min(), points[:, 1].max()
    x_pad = (x_max - x_min) * 0.05
    y_pad = (y_max - y_min) * 0.05
    ax.set_xlim(x_min - x_pad, x_max + x_pad)
    ax.set_ylim(y_min - y_pad, y_max + y_pad)

    plt.show()

plot_with_ngons(points_7, 7)
plot_with_ngons(points_8, 8)